In [ ]:
import insightface
import warnings
warnings.simplefilter('ignore')

In [ ]:
from insightface.app import FaceAnalysis
app_l=FaceAnalysis(name='buffalo_l',root='insightface_model')

In [ ]:
app_l.prepare(ctx_id=0,det_size=(640,640))

In [ ]:
import cv2
img=cv2.imread('insightface/test_image_1.jpg')
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
results_l=app_l.get(img)

In [ ]:
print(results_l)

In [ ]:
len(results_l)

In [ ]:
results_l[0].keys()

In [ ]:
results_l[0].kps

In [ ]:
results_l[0].det_score

In [ ]:
img_copy=img.copy()
for r in results_l:
    x1,y1,x2,y2=r['bbox'].astype('int')
    
    cv2.rectangle(img_copy,(x1,y1),(x2,y2),(0,255,0),1)
    
    kps=r['kps'].astype('int')
    for k1,k2 in kps:
        cv2.circle(img_copy,(k1,k2),2,(255,255,0),-1)
    
    score="score: {}%".format(int(r['det_score']*100))
    cv2.putText(img_copy,score,(x1,y1),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255))
    
    gender_en=['Female','Male']
    gender=gender_en[r['gender']]
    age=r['age']
    age_gender=f"{gender}::{age}"
    cv2.putText(img_copy,age_gender,(x2,y2),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255))
    
cv2.imshow('bbox',img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import re
import os
import cv2
person_info=[]
def clean_name(string):
    string=re.sub(r'[^A-Za-z]',' ',string)
    string=string.title()
    return string

path='D:\project\Facial Recognition\data\images'
listdir=os.listdir(path=path)
for folder in listdir:
    role, name= folder.split('-')
    name=clean_name(name)
    role=clean_name(role)
    #print(role,name)
    
    img_files=os.listdir(path=f'{path}/{folder}')
    #print(img_files)
    
    for file in img_files:
        p=f'{path}/{folder}/{file}'
        img=cv2.imread(p)
        result=app_l.get(img)
        if len(result)>0:
            res=result[0]
            embedding=res['embedding']
            person_info.append([name,role,embedding])

In [ ]:
import pandas as pd
df=pd.DataFrame(person_info,columns=['Name','Role','Facial_Features'])
df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def search(df,feature_col,test_vector,threshold=0.5):
    df=df.copy()
    x_list=df[feature_col].to_list()
    x=np.asarray(x_list)
    
    similar=cosine_similarity(x,test_vector.reshape(1,-1))
    similar_arr=np.array(similar).flatten()
    df['cosine']=similar_arr
    
    data_filter=df.query(f'cosine >= {threshold}')
    if len(data_filter)>0:
        data_filter.reset_index(drop=True,inplace=True)
        argmax=data_filter['cosine'].argmax()
        person_name,person_role=data_filter.loc[argmax][['Name','Role']]
        
    else:
        person_name='Unknown'
        person_role='Unknown'

    return person_name,person_role

In [ ]:
import cv2
test_image=cv2.imread('insightface/test_11.jpg')
cv2.imshow('test image',test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
results=app_l.get(test_image)
test_copy=test_image.copy()
import numpy as np
for r in results:
    x1,y1,x2,y2=r['bbox'].astype('int')
    embeddings=r['embedding']
    person_name,person_role=search(df,'Facial_Features',embeddings,threshold=0.5)
    
    if person_name=='Unknown':
        color=(0,0,255)
    else:
        color=(0,255,0)
    cv2.rectangle(test_copy,(x1,y1),(x2,y2),color) 
    cv2.putText(test_copy,person_name,(x1,y1),cv2.FONT_HERSHEY_DUPLEX,0.7,color,2)
    
cv2.imshow('test image',test_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
df_mean=df.groupby(by=['Name','Role']).mean()

In [ ]:
df_mean.reset_index(inplace=True)

In [ ]:
results=app_l.get(test_image)
test_copy=test_image.copy()
import numpy as np
for r in results:
    x1,y1,x2,y2=r['bbox'].astype('int')
    embeddings=r['embedding']
    person_name,person_role=search(df_mean,'Facial_Features',embeddings,threshold=0.5)
    
    if person_name=='Unknown':
        color=(0,0,255)
    else:
        color=(0,255,0)
    cv2.rectangle(test_copy,(x1,y1),(x2,y2),color) 
    cv2.putText(test_copy,person_name,(x1,y1),cv2.FONT_HERSHEY_DUPLEX,0.7,color,2)
    
cv2.imshow('test image',test_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
xvalues=df_mean.values
cols=np.array(df_mean.columns)
cols

In [ ]:
np.savez('df_manager_employees.npz',xvalues,cols)

In [ ]:
file_np=np.load('df_manager_employees.npz',allow_pickle=True)
file_np.files

In [ ]:
pd.DataFrame(file_np['arr_0'],columns=file_np['arr_1'])